In [7]:
%pip install langchainhub 
%pip install bs4
%pip install openllm
%pip install accelerator
%pip install litellm=1.24.0
%pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"
%pip install "ipython"
%pip install unstructured
%pip install llama_index
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-langchain
%pip install psycopg2-binary
%pip install pgvector

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached prompt_toolkit-3.0.36-py3-none-any.whl.metadata (7.0 kB)
Using cached prompt_toolkit-3.0.36-py3-none-any.whl (386 kB)
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.48
    Uninstalling prompt_toolkit-3.0.48:
      Successfully uninstalled prompt_toolkit-3.0.48
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.29.0 requires prompt-toolkit<3.1.0,>=3.0.41, but you have prompt-toolkit 3.0.36 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Invalid requirement: 'litellm=1.24.0': Expected end or semicolon (after name and no valid version specifier)
    litellm=

In [8]:
#Do the migration
import os
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.pgvector import PGVector
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import SimpleDirectoryReader, StorageContext
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import Settings

In [5]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')


CONNECTION_STRING = f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}'

In [6]:
##################### EXAMPLE EMBEDDINGS ############################
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text, "This is not a test document."])

/tmp/ipykernel_456/233229824.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Use utility script to download Act data
# VPN must be on and S3 keys set for this
os.system('python utility/s3.py')
# Import those files
documents = SimpleDirectoryReader("./Acts").load_data()

Traceback (most recent call last):
  File "/app/utility/s3.py", line 17, in <module>
    response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/botocore/client.py", line 1005, in _make_api_call
    http, parsed_response = self._make_request(
                            ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/botocore/client.py", line 1029, in _make_request
    return self._endpoint.make_request(operation_model, request_dict)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/botocore/endpoint.py", line 119, in make_request
    return self._send_request(request_d

In [8]:
print((len(documents)))

569


In [14]:
url = make_url(CONNECTION_STRING)
Settings.embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [15]:
vector_store = PGVectorStore.from_params(
    database=POSTGRES_DB,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="Acts",
    embed_dim=384,  # openai embedding dimension
)

In [16]:
from llama_index.core.node_parser import SentenceSplitter

In [17]:
text_parser = SentenceSplitter(
    chunk_size=512,
    # separator=" ",
)

In [18]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [19]:
print(len(doc_idxs))

23076


In [20]:
from llama_index.core.schema import TextNode
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [21]:
print(len(nodes))
print(nodes[0])

23076
Node ID: 3f2dc214-ca17-4ec2-8c69-7c242b10fc64
Text: Access to Abortion Services Act  [RSBC 1996] CHAPTER 1
Preamble      WHEREAS all people in British Columbia are entitled to
access to  health care, including abortion services;      AND WHEREAS
all people who use the British Columbia health care  system, and who
provide services for it, should be treated with  courtesy and with
resp...


In [22]:
# This can take a long time on local. (>80 mins)
for node in nodes:
    node_embedding = Settings.embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [23]:
print(nodes[0].embedding)
vector_store.add(nodes)

[-0.009175238199532032, 0.027766039595007896, -0.05048796907067299, -0.051523707807064056, -0.019916484132409096, 0.045855991542339325, 0.06473847478628159, -0.0018149953102692962, 0.05061834305524826, 0.044385332614183426, -0.027274660766124725, 0.0573023185133934, -0.06774342805147171, -0.10318140685558319, -0.011988447979092598, 0.097600057721138, 0.05839073285460472, -0.09239397943019867, -0.03387284278869629, 0.1647776961326599, 0.044537972658872604, 0.01195389311760664, 0.01425220537930727, 0.0555272214114666, -0.01276025827974081, -0.042324576526880264, -0.021250979974865913, -0.012486784718930721, -0.01640472747385502, 0.015190758742392063, 0.05044926702976227, -0.008084189146757126, 0.060854699462652206, 0.022693965584039688, 0.015014962293207645, 0.04506773501634598, 0.03234843909740448, -0.019008491188287735, 0.014289329759776592, 0.02386127971112728, -0.011289621703326702, 0.039754271507263184, -0.03327690809965134, 0.1408095508813858, -0.004725795239210129, 0.0001049179772

['3f2dc214-ca17-4ec2-8c69-7c242b10fc64',
 'f1a1f386-ff1f-4030-96e8-f4f78dce1171',
 '03b47832-ed71-4f53-b493-ef39d7e80bcb',
 '34d29e15-7e07-4db4-906b-e69e565234cd',
 '9d04ba69-009a-4fc5-8525-a2f30aacfa83',
 'a72bdbe1-9d6f-4a31-b42e-4cb4ded97232',
 'd35602fe-bb95-46a5-b376-9c89cfcb2229',
 '932f8e2d-024b-47f9-a83b-651470eefa67',
 '440baa31-18d3-4555-bcb3-ebfe493f9f94',
 '08e2b281-2a83-4bfe-8606-bf308dc61186',
 'dd7e2d1c-cfd1-4099-add9-42b0e70c2c7c',
 '9e590c3d-236b-449d-8894-0684da38d3e6',
 '27602d4d-c71e-4b90-a7ba-fbbd138a6bc3',
 '03e6c89e-0100-4824-8b23-a928ae1e285b',
 '53698692-ad2d-445b-be05-0e3336750710',
 '42e3c790-9929-4bc9-9e58-fcfd4d51e95d',
 'ce05f11d-b663-4866-b444-c2f4cf4a39d9',
 'eb2e5558-1d21-4182-b0cd-7af5aacbf475',
 '1a806f45-3ec4-4772-9773-7e5f0b662d34',
 '5b7cb8a1-b52c-4fae-9433-8b97d3ed249b',
 'ede587f8-419c-464c-920b-9bb8d0644987',
 'fcd9bb1e-21e7-40c6-9c61-384e9a8fe323',
 'bff1162c-47a5-4a22-b5b5-6089e579111e',
 '642d69e2-17ae-4219-ba1f-97bfa10a1c10',
 '83700142-1694-

In [24]:
query_str = "I've been served with an eviction notice by my landlord. How much notice are they required to give me under the law?"

In [25]:
query_embedding = Settings.embed_model.get_query_embedding(query_str)

In [26]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [27]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

(2)

If the tenant paid rent before giving a notice under subsection (1),
on receiving the tenant's notice, the landlord must refund any rent
paid for a period after the effective date of the tenant's notice.




(3)

A notice under this section does not affect the tenant's right to
compensation under section 51
[tenant's compensation: section 49 notice]
.






Tenant's compensation: section 49 notice




51

(1)


A tenant who receives a notice to end a tenancy under section 49
[landlord's use of property]
is entitled to receive from the landlord on or before the effective
date of the landlord's notice an amount that is the equivalent of one
month's rent payable under the tenancy agreement.




(1.1)

A tenant referred to in subsection (1) may withhold the amount
authorized from the last month's rent and, for the purposes of section
50 (2), that amount is deemed to have been paid to the landlord.




(1.2)

If a tenant referred to in subsection (1) paid rent before giving a
notice un

In [28]:
def retrieval(query_str):
    query_embedding = Settings.embed_model.get_query_embedding(query_str)
    query_mode = "default"
    # query_mode = "sparse"
    # query_mode = "hybrid"
    
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)
    #print(query_result.nodes[0].get_content())
    return query_result.nodes[0].get_content()


In [9]:
## NOTE: Cannot progress past this point without fixing issue with trulens import
%pip uninstall -y trulens_eval
%pip install trulens-eval==0.25.1

from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
TRULENS_USER = os.getenv('TRULENS_USER')
TRULENS_PASSWORD = os.getenv('TRULENS_PASSWORD')
TRULENS_DB = os.getenv('TRULENS_DB')
TRULENS_PORT = os.getenv('TRULENS_PORT')
TRULENS_HOST = os.getenv('TRULENS_HOST')

TRULENS_CONNECTION_STRING = f'postgresql+psycopg2://{TRULENS_USER}:{TRULENS_PASSWORD}@{TRULENS_HOST}:{TRULENS_PORT}/{TRULENS_DB}'
tru = Tru(database_url=TRULENS_CONNECTION_STRING)

Found existing installation: trulens_eval 0.25.1
Uninstalling trulens_eval-0.25.1:
  Successfully uninstalled trulens_eval-0.25.1
Note: you may need to restart the kernel to use updated packages.
  Using cached trulens_eval-0.25.1-py3-none-any.whl.metadata (3.3 kB)
Using cached trulens_eval-0.25.1-py3-none-any.whl (673 kB)
Note: you may need to restart the kernel to use updated packages.


Package litellm is installed but has a version conflict:
	(litellm 1.51.2 (/usr/local/lib/python3.11/site-packages), Requirement.parse('litellm<=1.24.0,>=1.11.1'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'litellm<=1.24.0,>=1.11.1'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```



KeyError: '__name__'

In [3]:
tru.reset_database()

NameError: name 'tru' is not defined

In [1]:
import openllm

llm = openllm.LLM(
    "google/flan-t5-small",
    temperature=0.94,
    repetition_penalty=1.2,

)

AttributeError: module 'openllm' has no attribute 'LLM'

In [ ]:
import asyncio

import nest_asyncio



nest_asyncio.apply()


async def main(query):
  previous_texts = ''  
  async for gen in llm.generate_iterator(query, max_new_tokens=128):
      print(gen.outputs[0].text, flush=True, end='')
      previous_texts += gen.outputs[0].text
  return previous_texts


asyncio.run(main("what is tenancy act. Please explain"))

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return retrieval(query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        messages=f"We have provided context information below. \n" \
            f"---------------------\n" \
            f"{context_str}" \
            f"\n---------------------\n" \
            f"Given this information, please answer the question: {query}"
        print(messages)
        completion = asyncio.run(main(messages))
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [ ]:

rag.query("When was the University of Washington founded?")



In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    #feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance]
    )


In [ ]:
with tru_rag as recording:
    rag.query("I've been served with an eviction notice by my landlord. How much notice are they required to give me under the law?")

In [ ]:
from ipywidgets import Button, HBox, VBox
from IPython.display import display

thumbs_up_button = Button(description='👍')
thumbs_down_button = Button(description='👎')

human_feedback = None

def on_thumbs_up_button_clicked(b):
    global human_feedback
    human_feedback = 1

def on_thumbs_down_button_clicked(b):
    global human_feedback
    human_feedback = 0

thumbs_up_button.on_click(on_thumbs_up_button_clicked)
thumbs_down_button.on_click(on_thumbs_down_button_clicked)

HBox([thumbs_up_button, thumbs_down_button])

In [ ]:
record = recording.get()
print(human_feedback)
tru.add_feedback(
    name="Human Feedack",
    record_id=record.record_id,
    app_id=tru_rag.app_id,
    result=human_feedback
)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.tail(5)

In [ ]:

tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard(address='0.0.0.0', force=True)

In [ ]:


tru.get_leaderboard(app_ids=[tru_rag.app_id])
